In [1]:
from get_data import create_directories, build_search_url, download_extended_page, download_image, get_all_items

In [4]:
sufix = "face"

keywords = ['omar sy', 'elon musk']
keywords = [keyword + " " + sufix for keyword in keywords]
search_keywords = [str(item) for item in keywords]

limit = 200

main_directory = "downloads"

In [5]:
i_search_keywoard = 0

while i_search_keywoard < len(search_keywords):
	search_term = search_keywords[i_search_keywoard]

	subdirectory_name = search_term
	create_directories(main_directory, subdirectory_name)

	url = build_search_url(search_term)
	raw_html = download_extended_page(url)

	get_all_items(raw_html, main_directory, subdirectory_name, limit)

	i_search_keywoard += 1

We created the directories.




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\louis\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


https://www.google.com/search?q=omar%20sy%20face&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg
Getting you a lot of images. This may take a few moments...
Reached end of Page.




====== WebDriver manager ======


We created the directories.


Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\louis\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


https://www.google.com/search?q=elon%20musk%20face&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg
Getting you a lot of images. This may take a few moments...
Reached end of Page.
Error
